In [3]:
import pandas as pd 
import requests
import json
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import urllib.request
from PIL import Image
import selenium
#import langchain

In [4]:
recipe_page = "https://pinchofyum.com/salmon-tacos"

In [5]:
page = requests.get(recipe_page)
food_pages_soup = BeautifulSoup(page.text, 'html.parser')

### Parse Single Recipe Page

In [ ]:
def pinch_of_yum_nutrients(soup):
    "Nutrients are rendered with JS - would need selenium"
    pass

In [33]:
food_pages_soup.find("div",attrs={"class":"tasty-recipes-nutrifox"}).findAll("div",attrs={"class":"label-nutrient"})

[]

In [23]:
def pinch_of_yum_ingredients(soup):
    li_objs = soup.find("div",attrs={"class":"tasty-recipes-ingredients"}).findAll("li")
    ingredients = []
    for li_obj in li_objs: 
        if li_obj.find("span",attrs={"class":"nutrifox-quantity"}):
            ingredients.append({
                "amt":li_obj.find("span",attrs={"class":"nutrifox-quantity"}).text,
                "unit":li_obj.find("span",attrs={"class":"nutrifox-unit"}).text,
                "ingred":li_obj.find("strong").text,
            })

    return ingredients

In [24]:
def parse_pinch_of_yum_page(url):
    page = requests.get(url)
    food_pages_soup = BeautifulSoup(page.text, 'html.parser')
    recipe_obj = {
        "title":food_pages_soup.find("h1",attrs={"class":"entry-title mb-2"}).text,
        "url":food_pages_soup.find("div", attrs={"class":"tasty-pins-banner-container"}).find("img")["src"],
        "ingredients": pinch_of_yum_ingredients(food_pages_soup),
        "instructions":[itm.text.replace("\xa0","") for itm in food_pages_soup.find("div",attrs={"class":"tasty-recipes-instructions"}).findAll("li")],
        "prep_time":food_pages_soup.find("li",attrs={"class":"prep-time"}).text.replace("Prep Time: ",""),
        "cook_time":food_pages_soup.find("li",attrs={"class":"cook-time"}).text.replace("Cook Time: ",""),
        "category":food_pages_soup.find("li",attrs={"class":"category"}).text.replace("Category: ",""),
        "cuisine":food_pages_soup.find("li",attrs={"class":"cuisine"}).text.replace("Cuisine: ",""),
    }
    return recipe_obj

In [25]:
parse_pinch_of_yum_page("https://pinchofyum.com/summer-chipotle-chicken-cobb-salad-with-cilantro-vinaigrette")

{'title': 'Summer Chipotle Chicken Cobb Salad with Cilantro Vinaigrette',
 'url': 'https://pinchofyum.com/wp-content/uploads/Summer-Cobb-Salad-in-Bowl.jpg',
 'ingredients': [{'amt': '3', 'unit': 'cups', 'ingred': 'greens'}],
 'instructions': ['Prep the chicken: Combine the oil, chipotles, garlic powder, cumin, oregano, black pepper, and salt in a bowl. Place the chicken in the bowl and toss to coat the chicken in the marinade. Place in the fridge and let it marinate while you prep your other ingredients.',
  'Prep salad: Prep all your salad ingredients (cook the bacon, slice the strawberries, cut the corn off the cob, etc.).',
  'Grill the chicken: Heat an outdoor grill (or grill pan) to about 400 degrees. Place chicken on the grill and grill for 5-6 minutes per side, until cooked through. Remove the chicken from the grill and let rest for ten minutes. Slice against the grain.',
  'Salad: Arrange the greens on a large platter and top with the bacon, strawberries, corn, avocado, and gri

### Crawler

In [26]:
root_url = "https://pinchofyum.com/recipes"

In [27]:
page_root = "https://pinchofyum.com/"

In [28]:
recipe_objs = []
nav_links = ["https://pinchofyum.com/recipes/quick-and-easy"]
recipe_links = []
scraped = set()
while (nav_links or recipe_links) and len(recipe_objs) <500:
    if recipe_links:
        link = recipe_links.pop()
    else:
        link = nav_links.pop()
    print(f"Parsing {link}")
    scraped.add(link)
    page = BeautifulSoup(requests.get(link).text, 'html.parser')

    if page.find("div",attrs={"class":"tasty-recipes-entry-content"}):
        print("adding recipe")
        try:
            recipe_objs.append(parse_pinch_of_yum_page(link))
        except AttributeError:
            print(f"Error scraping {link}. Skipping")
    
    else:
        all_links = [l["href"] for l in page.findAll("a") if l.has_attr("href")]
        all_links = list(set(all_links))
        for link in all_links:
            if link not in scraped and "comment" not in link:
                if page_root in link and root_url not in link:
                    recipe_links.append(link)
                elif root_url in link:
                    nav_links.append(link)



Parsing https://pinchofyum.com/recipes/quick-and-easy
Parsing https://pinchofyum.com/instant-pot-mac-and-cheese
adding recipe
Parsing https://pinchofyum.com/garlic-butter-baked-penne
adding recipe
Parsing https://pinchofyum.com/roasted-broccoli-salad
adding recipe
Parsing https://pinchofyum.com/terms-of-service
Parsing https://pinchofyum.com/blog
Parsing https://pinchofyum.com/blog/page/154
Parsing https://pinchofyum.com/zucchini-fritters-goat-cheese-sauce
adding recipe
Error scraping https://pinchofyum.com/zucchini-fritters-goat-cheese-sauce. Skipping
Parsing https://pinchofyum.com/tomato-ricotta-pasta-with-broccoli
adding recipe
Parsing https://pinchofyum.com/media-mentions
Parsing https://pinchofyum.com/start-here
Parsing https://pinchofyum.com/15-minute-meal-prep-sesame-noodle-bowls
adding recipe
Parsing https://pinchofyum.com/instant-pot-wild-rice-soup
adding recipe
Parsing https://pinchofyum.com/chopped-thai-chicken-salad
adding recipe
Parsing https://pinchofyum.com/instant-pot-r

In [30]:
recipe_objs[0]

{'title': 'Instant Pot Mac and Cheese',
 'url': 'https://pinchofyum.com/wp-content/uploads/Best-Instant-Pot-Mac-and-Cheese.jpg',
 'ingredients': [{'amt': '1', 'unit': 'pound', 'ingred': 'elbow pasta'},
  {'amt': '4', 'unit': 'cups', 'ingred': 'water'},
  {'amt': '2 1/2', 'unit': 'cups', 'ingred': 'shredded cheese'},
  {'amt': '1/2', 'unit': 'cup', 'ingred': 'whole milk '}],
 'instructions': ['Noodles: Put the pasta, water, and salt into the Instant Pot. Cook for 4 minutes using the manual function. Quick release the steam so your noodles don’t get overcooked.',
  'Cheese: Gently stir in the butter until melted. Add the cheese and milk (use 2-3 tablespoons of milk at a time, as needed, up to 1/2 cup). Add more seasoning or liquid if necessary. For best results, let it sit for 5-10 minutes to let it thicken up a bit. MWAH! Enjoy. It’s so, so good.'],
 'prep_time': '5 minutes',
 'cook_time': '5 minutes',
 'category': 'Dinner',
 'cuisine': 'American'}

In [31]:
recipe_objs[1]

{'title': 'No-Boil Baked Penne with Meatballs',
 'url': 'https://pinchofyum.com/wp-content/uploads/Baked-Penne-3.jpg',
 'ingredients': [],
 'instructions': ['Preheat oven to 400 degrees.',
  'Mix all of the baked penne ingredients together in a 9×13 baking dish. Cover tightly with foil (double layer isn’t a bad idea).',
  'Bake for 35-40 minutes.',
  'Remove foil (HOT HOT HOT) and give it a stir. Add spinach and top with Parm, and brown it up for 3-5 minutes under the broiler. Let it stand for a few minutes so the sauce has a chance to thicken up.Voila!'],
 'prep_time': '10 mins',
 'cook_time': '45 mins',
 'category': 'Dinner',
 'cuisine': 'Italian'}

In [32]:
with open("../../../../hackathon-mvp/prisma/data/recipes_seed2.json","w") as f:
    json.dump({"recipes":recipe_objs},f)

In [78]:
'https://pinchofyum.com/recipes/quick-and-easy' in scraped

True

In [54]:
page = BeautifulSoup(requests.get(link).text, 'html.parser')

In [64]:
all_links = [l["href"] for l in page.findAll("a") ]

In [65]:
all_links

['https://pinchofyum.ck.page/3527a72463',
 'https://pinchofyum.com',
 'https://pinchofyum.com',
 'https://pinchofyum.com/about',
 'https://pinchofyum.com/recipes',
 'https://pinchofyum.com/start-here',
 'https://pinchofyum.com/?s=',
 'https://pinchofyum.com/',
 'https://pinchofyum.com/recipes',
 'https://pinchofyum.com/garlic-butter-shrimp-boil',
 'https://pinchofyum.com/garlic-butter-shrimp-boil',
 'https://pinchofyum.com/house-favorite-brussels-sprouts',
 'https://pinchofyum.com/spaghetti-with-crispy-zucchini',
 'https://pinchofyum.com/broccoli-cheese-soup',
 'https://pinchofyum.com/basic-soft-pretzels',
 'https://pinchofyum.com/chickpea-curry',
 'https://pinchofyum.com/roasted-broccoli-salad',
 'https://pinchofyum.com/5-ingredient-spicy-pork',
 'https://pinchofyum.com/harissa-meatballs-with-whipped-feta',
 'https://pinchofyum.com/sheet-pan-meatballs-with-tomato-salad-and-green-sauce',
 'https://pinchofyum.com/caprese-grilled-cheese',
 'https://pinchofyum.com/trader-joes-sun-dried-to

In [60]:
len(page.findAll("a"))

155

In [63]:
page.findAll("a")[100]

False